In [26]:
import os
import cv2
import numpy as np
import pandas as pd
import sys
sys.path.append("..")
from utils.imutils import jimshow as show
from utils.imutils import jimshow_channel as show_channel
import matplotlib.pyplot as plt

In [42]:
 # Calculate histogram for each image and store it
 def compare_histograms(in_folderpath, image_files, chosen_normalized_hist):

    image_names = []
    results = []

    for image_file in image_files:
        image_path = os.path.join(in_folderpath, image_file)
        image_names.append(image_file)
        image = cv2.imread(image_path)
        image_hist = cv2.calcHist([image], [0,1,2], None, [256,256,256], [0,256, 0,256, 0,256])
        normalized_hist = cv2.normalize(image_hist, image_hist, 0, 1.0, cv2.NORM_MINMAX)
        comparisons = round(cv2.compareHist(chosen_normalized_hist, normalized_hist, cv2.HISTCMP_CHISQR),2)
        results.append(comparisons)


In [28]:
def save_results(in_folderpath, out_folderpath, image_names, results):

    df = pd.DataFrame({'Filename': image_names,'Distance': results})

    df_sorted = df.sort_values(by=['Distance'], ascending=True).head(6)

    # print(df_sorted['Distance'].max())
    
    # for i in range(len(df_sorted)):
    #     image_path = os.path.join(in_folderpath, df_sorted.iloc[i]['Filename'])
    #     image = cv2.imread(image_path)
    #     show(image)

    df_sorted.to_csv(os.path.join(out_folderpath, 'image_comparisons.csv'), index=False)

In [43]:
def main(in_folderpath, out_folderpath, image_index):
    # Creates a sorted list of all the images within the given folder path
    image_files = sorted(os.listdir(os.path.join(in_folderpath)))
    chosen_image_path = os.path.join(in_folderpath, image_files[image_index])
   
    # image_names = []
    # results = []

    chosen_image = cv2.imread(chosen_image_path)
    chosen_image_hist = cv2.calcHist([chosen_image], [0,1,2], None, [256,256,256], [0,256, 0,256, 0,256])
    chosen_normalized_hist = cv2.normalize(chosen_image_hist, chosen_image_hist, 0, 1.0, cv2.NORM_MINMAX)
    
    compare_histograms(in_folderpath, image_files, chosen_normalized_hist)
    
    # Calculate histogram for each image and store it
    # for image_file in image_files:
    #     image_path = os.path.join(in_folderpath, image_file)
    #     image_names.append(image_file)
    #     image = cv2.imread(image_path)
    #     image_hist = cv2.calcHist([image], [0,1,2], None, [256,256,256], [0,256, 0,256, 0,256])
    #     normalized_hist = cv2.normalize(image_hist, image_hist, 0, 1.0, cv2.NORM_MINMAX)
    #     comparisons = round(cv2.compareHist(chosen_normalized_hist, normalized_hist, cv2.HISTCMP_CHISQR),2)
    #     results.append(comparisons)

    save_results(in_folderpath, out_folderpath, image_names, results)


if __name__ == "__main__":
    main("../in", "../out", 1356)

NameError: name 'image_names' is not defined